* 형태소 분석 (문어/구어) : "morp", -> 사용할 단위
* 어휘의미 분석 (동음이의어 분석)(문어) : "wsd"
* 어휘의미 분석 (다의어 분석)(문어) : "wsd_poly"
* 개체명 인식 (문어/구어) : "ner"
* 의존 구문 분석 (문어) : "dparse"
* 의미역 인식 (문어) : "srl"

# upload data

In [1]:
import pandas as pd
import urllib3
import json

df1 = pd.read_json('/home/user_1/medistream-recsys/Script/YS/df_book_clean.json')
df = df1.copy()

In [3]:
api_key = '8e216d4f-bfae-4cc4-ae6f-25f2dc4968c5'

언어 분석 기술 문어/구어 중 한가지만 선택해 사용

In [4]:
# // 언어 분석 기술(문어)
openApiURL = "http://aiopen.etri.re.kr:8000/WiseNLU" 
# // 언어 분석 기술(구어)
# openApiURL = "http://aiopen.etri.re.kr:8000/WiseNLU_spoken"

# preprocessing

### 1. api로 불러온 data에서 필요한 정보 추출

In [5]:
class ETRISentenceAnalysis:
    def __init__(self, api_key: dict, url: str):
        self.api_key = api_key
        self.url = url

        self.http = urllib3.PoolManager()
  
    def _make_request_json(self, text: str, analysis_code: int) -> dict:
        return {
          "access_key": self.api_key,
          "argument": {
              "text": text,
              "analysis_code": analysis_code
              }
          }

    def _request(self, request_json: dict) -> object:
        return self.http.request(
            "POST",
            openApiURL,
            headers={"Content-Type": "application/json; charset=UTF-8"},
            body=json.dumps(request_json)
        )

    def get_analyzed_sentence(self, sentence: str, analysis_code: int) -> dict:
        def _get_return_object(sentence: str) -> dict:
            data = str(response.data, 'utf-8')
            data = json.loads(data)


            return data.get('return_object') 

        request_json = self._make_request_json(sentence, analysis_code)
        response = self._request(request_json)

        assert response.status == 200, f'Error {response.status}'

        return _get_return_object(response)

In [6]:
def get_dependency_from_text(data: dict) -> dict:
    analyzed_sentences = data.get('sentence')
    
    extracted_sentences_by_text_and_dependency = {
        i: {'text':sentence.get('text'), # 문장 한개씩 뱉음
            'dependency': {
                'subject': [info.get('text') for info in sentence.get('dependency') if info.get('label') == 'NP' or info.get('label') == 'NP_SBJ'],
                'object': [info.get('text') for info in sentence.get('dependency') if info.get('label') == 'NP_OBJ']
                }
            } for i, sentence in enumerate(analyzed_sentences)
        }
    
    return extracted_sentences_by_text_and_dependency

In [7]:
# get only nouns
def get_nouns_from_text(data: dict) -> dict:
    analyzed_sentences = data.get('sentence')
    
    extracted_sentences_by_text_and_dependency = {
        i: {'text':sentence.get('text'), # 문장 한개씩 뱉음
            'dependency': [info.get('text') for info in sentence.get('morp') if info.get('type') == 'NNG']
            } for i, sentence in enumerate(analyzed_sentences)
        }
    
    return extracted_sentences_by_text_and_dependency

In [8]:
etri_sentence_analysis = ETRISentenceAnalysis(api_key, openApiURL)

### 2. 1만 글자 이하로 전처리(1만 글자가 넘는 경우 api가 처리하지 못함)

In [4]:
# 1만 글자 넘는 경우 확인
# for i in range(len(df)):
#     if len(df['description'].iloc[i]) >= 10000:
#         print(df.iloc[i])
        
# 1만 글자 넘는 것은 띄어쓰기 없애는 전처리 진행
df['description'][df.index == 126] = df[df.index == 126].description.values[0].replace(" ",'')
df['description'][df.index == 229] = df[df.index == 229].description.values[0].replace(" ",'')    

<ipython-input-4-c2177851f8b7>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['description'][df.index == 126] = df[df.index == 126].description.values[0].replace(" ",'')
<ipython-input-4-c2177851f8b7>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['description'][df.index == 229] = df[df.index == 229].description.values[0].replace(" ",'')


# corpus에 명사만 담기

In [5]:
# corpus에 형태소 분석 결과 담기

corpus = []
for i in range(200):
    con = df['description'].iloc[i]
    # 형태소 분석
    data = etri_sentence_analysis.get_analyzed_sentence(con, 'morp')
    corpus.append(data)
len(corpus)

NameError: name 'etri_sentence_analysis' is not defined

In [ ]:
# dataframe 초기화
new_df = pd.DataFrame(df[['name_x','description']])
new_df['tokens'] = None

# dataframe 각 토픽 담기
for i in range(len(corpus)):
    nouns = []
    res3 = corpus[i]['sentence'][0]['morp'] # 형태소 결과
    
    for j in res3:
        if j['type'] == 'NNG' and len(j['lemma']) > 1: #  형태소가 명사이고 2글자 이상인 것만
            nouns.append(j['lemma'])
            new_df['tokens'].iloc[i] = nouns

new_df

In [ ]:
# new_df.loc[:199,].to_json('/home/user_4/CBF/tokens_199.json')

In [17]:
# df.loc[200:,].to_json('/home/user_4/CBF/to200frame.json')